# PEGG 2.0

This is the real notebook for making the next generation of PEGG...

- Start with the flexible input + getting it to work WITH error checking, etc...
- I should translate everything into the easiest possible format to work with that is the most flexible...


- To do:
    - Get the flexible input format in PrimeDesign format working as well!

In [67]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import Bio.Seq
import gzip
from Bio import SeqIO
from Bio.Align import PairwiseAligner
from Bio.pairwise2 import format_alignment
import warnings
import re
warnings.filterwarnings('ignore')

In [35]:
MATCH_SCORE = 1
MISMATCH_SCORE = -0.5
OPEN_GAP_SCORE = -5
EXTEND_GAP_SCORE = -0.1
TARGET_END_GAP_SCORE = 0
QUERY_END_GAP_SCORE = 0

def make_aligner():
    aligner = PairwiseAligner()
    aligner.mode = 'global'
    aligner.match_score = MATCH_SCORE
    aligner.mismatch_score = MISMATCH_SCORE
    aligner.open_gap_score = OPEN_GAP_SCORE
    aligner.extend_gap_score = EXTEND_GAP_SCORE
    aligner.target_end_gap_score = TARGET_END_GAP_SCORE
    aligner.query_end_gap_score = QUERY_END_GAP_SCORE
    return aligner

aligner = make_aligner()

In [2]:
class mutation:
    def __init__(self, forward_sequence, mut_forward_sequence, chrom=None, genome=None):
        """ 
        change seq_start and seq_end to correspond with chromosome coordinates
        or some other organizational method...
        """
        
        #WT sequence
        #enforce uppercase
        self.forward_sequence = forward_sequence.upper()
        rc = str(Bio.Seq.Seq(self.forward_sequence).reverse_complement())
        self.rev_comp_sequence = rc

        #seq start refers to the WIDE sequence start (i.e. the WT sequence with e.g. 60 nt of context)
        self.seq_start = 0
        self.seq_end = len(self.forward_sequence)
        self.chrom = chrom
        self.genome_build = genome

        #mutation information
        self.mut_forward_sequence = mut_forward_sequence.upper()
        rc_mut = str(Bio.Seq.Seq(self.mut_forward_sequence).reverse_complement())
        self.mut_rev_comp_sequence = rc_mut

        #self.variant_type = 
        #self.ref = 
        #self.alt = 
        #self.alt_size = (positive or negative integer)
        

        #information about the PAM sequence location
        self.PAM_idx_forward = None
        self.PAM_idx_rev_comp = None

In [6]:
#-----------functions------------
def genome_loader(filepath_gz):
    """
    Takes in filepath of human genome (GrCH37 or GrCh38) and returns records and index_list for PEGG parsing.
    
    Parameters
    -----------
    filepath_gz
        *type = str*
        
        The filepath to the .gz file holding the reference genome file.
    
    """
    #------loading in reference genome and organizing it into a 2-d list by chromosome---------------------
    wrong = ["alternate", "unplaced", "unlocalized", "patch", "mitochondrion"]
    filtered = []
    chrom_list = []
    seq_list = []

    with gzip.open(filepath_gz, "rt") as handle:
        #records = list(SeqIO.parse(handle, "fasta")) #about 4 Gb in  memory

        for i in SeqIO.parse(handle, "fasta"):
            ii = i.description
            ignore=False 

            for key in wrong:
                if key in ii:
                    ignore = True
            
            if ignore==False:

                x = ii.find('chromosome')
                chrom = ii[x:].split(' ')[1]
                chrom = chrom.split(',')[0]
                if chrom not in ['X', 'Y']:
                    chrom = int(chrom)
                chrom_list.append(chrom)
                seq_list.append(i.seq)

                filtered.append(ii)

  
    return dict(zip(chrom_list, seq_list)), filtered

In [13]:
filepath_37 = '/Users/samgould/Desktop/FSR Lab/reference files/GRCh37/ncbi-genomes-2022-03-17/GCF_000001405.25_GRCh37.p13_genomic.fna.gz'
filepath_38 = '/Users/samgould/Desktop/FSR Lab/reference files/GRCh38/ncbi-genomes-2022-02-23/GCF_000001405.26_GRCh38_genomic.fna.gz'
mouse = '/Users/samgould/Desktop/FSR Lab/reference files/GRCm38.p6 (mouse)/ncbi-genomes-2022-04-04/GCF_000001635.26_GRCm38.p6_genomic.fna.gz'
chrom_dict, i = genome_loader(filepath_37)

In [58]:
#starting by bringing in the code for extracting sequence

#need to modify this so that it works more generally (the chromosome shit in particular...)

def df_formatter(df, chrom_dict, context_size = 60, 
                 cols_to_save = ['Hugo_Symbol', 'Chromosome', 'Start_Position', 'End_Position', 'Consequence', 'Variant_Classification', 'Variant_Type', 'Reference_Allele', 'Tumor_Seq_Allele2',  'HGVSc', 'HGVSp_Short']):

    """ 
    Takes in variants (in cBioPortal format!)
    and outputs dataframe with WT and ALT oligos with designated context_size
    context_size = the amount of nt on either side of the variant e.g. AAA(A/G)AAA = context_size of 3
    cols_to_save = information to save about the mutations...(can modify beyond defaults...)
    """

    wt_w_context = []
    alt_w_context = []

    seq_start = []
    seq_end = []

    for i, val in df.iterrows():
        vt = val['Variant_Type']
        s = val['Start_Position']
        e = val['End_Position']
        ref = val['Reference_Allele']
        alt = val['Tumor_Seq_Allele2']
        chrom = val['Chromosome']

        if chrom not in ['X','Y']:
            chrom = int(chrom)
       
        chr_seq = chrom_dict[chrom].upper()

        if vt in ['SNP', 'ONP', 'DNP']:
            ref = ref
            alt = alt
            #assert ref == chr_seq[s-1:e], print(ref, chr_seq[s-1:e])
            left_context = chr_seq[s-1-context_size:s-1]
            right_context = chr_seq[e:e+context_size]

        elif vt =='INS':
            ref = ''
            alt = alt
            #left_context = chr_seq[s-1-context_size:s+1] #need to do this since INS reference alleles are blank
            left_context = chr_seq[s-1-context_size:s]
            right_context = chr_seq[e-1:e+context_size]

        elif vt=='DEL':
            ref = ref
            alt = ''
            left_context = chr_seq[s-1-context_size:s-1]
            right_context = chr_seq[e:e+context_size]

        wt_seq = left_context + ref + right_context
        alt_seq = left_context + alt + right_context

        wt_w_context.append(str(wt_seq))
        alt_w_context.append(str(alt_seq))

        start = s-context_size
        end = e+context_size

        seq_start.append(start)
        seq_end.append(end)

        assert str(chr_seq[start-1:end])==str(wt_seq), print(chr_seq[start-1:end] + '\n' + str(wt_seq))
                                                            

    df_new = df[cols_to_save]

    df_new['seq_start'] = seq_start
    df_new['seq_end'] = seq_end
    df_new['wt_w_context'] = wt_w_context
    df_new['alt_w_context'] = alt_w_context
    df_new = df_new.reset_index()
    
    return df_new


In [20]:
impact = pd.read_csv('/Users/samgould/Desktop/FSR Lab/reference files/2020-06-16-MSK-IMPACT_EDITED.txt', sep='\t')


/var/folders/5g/xxjcy2g531n02hnyb6d8qdsr0000gn/T/ipykernel_64096/4206583788.py:1: DtypeWarning: Columns (45,48,88) have mixed types. Specify dtype option on import or set low_memory=False.
  impact = pd.read_csv('/Users/samgould/Desktop/FSR Lab/reference files/2020-06-16-MSK-IMPACT_EDITED.txt', sep='\t')


In [59]:
p53 = impact[impact['Hugo_Symbol']=='TP53'].drop_duplicates(subset='HGVSc')

dels1 = p53[p53['Variant_Type']=='DEL']
bad_del_idx = list(dels1[dels1['Tumor_Seq_Allele2']!='-'].index)


ins1 = p53[p53['Variant_Type']=='INS']
bad_ins_idx = list(ins1[ins1['Reference_Allele']!='-'].index)

bad_idxs = bad_del_idx + bad_ins_idx

p53_filtered = p53.drop(index=bad_idxs).reset_index().drop(columns=['index'])

p53_formatted = df_formatter(p53_filtered, chrom_dict)

In [126]:
p53_formatted[p53_formatted['Variant_Type']=='DEL']

,index,Hugo_Symbol,Chromosome,Start_Position,End_Position,Consequence,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele2,HGVSc,HGVSp_Short,seq_start,seq_end,wt_w_context,alt_w_context
257,257,TP53,17,7578398,7578398,frameshift_variant,Frame_Shift_Del,DEL,G,-,ENST00000269305.4:c.532delC,p.H178Tfs*69,7578338,7578458,AGCCCTGTCGTCTCTCCAGCCCCAGCTGCTCACCATCGCTATCTGA...,AGCCCTGTCGTCTCTCCAGCCCCAGCTGCTCACCATCGCTATCTGA...
258,258,TP53,17,7578217,7578240,inframe_deletion,In_Frame_Del,DEL,GTGTTTCTGTCATCCAAATACTCC,-,ENST00000269305.4:c.609_632del,p.E204_T211del,7578157,7578300,ACCCCAGTTGCAAACCAGACCTCAGGCGGCTCATAGGGCACCACCA...,ACCCCAGTTGCAAACCAGACCTCAGGCGGCTCATAGGGCACCACCA...
270,270,TP53,17,7578206,7578207,frameshift_variant,Frame_Shift_Del,DEL,TA,-,ENST00000269305.4:c.642_643delTA,p.H214Qfs*7,7578146,7578267,TCCTCCCAGAGACCCCAGTTGCAAACCAGACCTCAGGCGGCTCATA...,TCCTCCCAGAGACCCCAGTTGCAAACCAGACCTCAGGCGGCTCATA...
274,274,TP53,17,7578237,7578253,frameshift_variant,Frame_Shift_Del,DEL,CTCCACACGCAAATTTC,-,ENST00000269305.4:c.596_612del,p.G199Vfs*4,7578177,7578313,CTCAGGCGGCTCATAGGGCACCACCACACTATGTCGAAAAGTGTTT...,CTCAGGCGGCTCATAGGGCACCACCACACTATGTCGAAAAGTGTTT...
276,276,TP53,17,7577525,7577527,inframe_deletion,In_Frame_Del,DEL,GAG,-,ENST00000269305.4:c.754_756del,p.L252del,7577465,7577587,AGGCCAGTGTGCAGGGTGGCAAGTGGCTCCTGACCTGGAGTCTTCC...,AGGCCAGTGTGCAGGGTGGCAAGTGGCTCCTGACCTGGAGTCTTCC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,2996,TP53,17,7579319,7579320,frameshift_variant,Frame_Shift_Del,DEL,GT,-,ENST00000269305.4:c.367_368del,p.T123Lfs*25,7579259,7579380,GATACGGCCAGGCATTGAAGTCTCATGGAAGCCAGCCCCTCAGGGC...,GATACGGCCAGGCATTGAAGTCTCATGGAAGCCAGCCCCTCAGGGC...
2999,2999,TP53,17,7579535,7579556,frameshift_variant,Frame_Shift_Del,DEL,TCAATATCGTCCGGGGACAGCA,-,ENST00000269305.4:c.131_152del,p.M44Nfs*72,7579475,7579616,GGAGCAGCCTCTGGCATTCTGGGAGCTTCATCTGGACCTGGGTCTT...,GGAGCAGCCTCTGGCATTCTGGGAGCTTCATCTGGACCTGGGTCTT...
3000,3000,TP53,17,7577539,7577541,inframe_deletion,In_Frame_Del,DEL,GGT,-,ENST00000269305.4:c.740_742del,p.N247del,7577479,7577601,GGTGGCAAGTGGCTCCTGACCTGGAGTCTTCCAGTGTGATGATGGT...,GGTGGCAAGTGGCTCCTGACCTGGAGTCTTCCAGTGTGATGATGGT...
3002,3002,TP53,17,7577531,7577532,frameshift_variant,Frame_Shift_Del,DEL,GG,-,ENST00000269305.4:c.749_750delCC,p.P250Hfs*13,7577471,7577592,GTGTGCAGGGTGGCAAGTGGCTCCTGACCTGGAGTCTTCCAGTGTG...,GTGTGCAGGGTGGCAAGTGGCTCCTGACCTGGAGTCTTCCAGTGTG...


In [127]:
i=257
alignments = aligner.align(p53_formatted.iloc[i]['wt_w_context'], p53_formatted.iloc[i]['alt_w_context'])
print(alignments[0])

AGCCCTGTCGTCTCTCCAGCCCCAGCTGCTCACCATCGCTATCTGAGCAGCGCTCATGGTGGGGGCAGCGCCTCACAACCTCCGTCATGTGCTGTGACTGCTTGTAGATGGCCATGGCGCG
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||-||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
AGCCCTGTCGTCTCTCCAGCCCCAGCTGCTCACCATCGCTATCTGAGCAGCGCTCATGGT-GGGGCAGCGCCTCACAACCTCCGTCATGTGCTGTGACTGCTTGTAGATGGCCATGGCGCG



In [132]:
WT_align = alignments[0].aligned[0]
alt_align = alignments[0].aligned[1]

print(WT_align)
print(alt_align)

((0, 60), (61, 121))
((0, 60), (60, 120))


In [133]:
WT_align

((0, 60), (61, 121))

In [134]:
s1 = p53_formatted.iloc[i]['wt_w_context']

print(s1[WT_align[0][0]:WT_align[0][1]])
print(s1[WT_align[1][0]:WT_align[1][1]])

AGCCCTGTCGTCTCTCCAGCCCCAGCTGCTCACCATCGCTATCTGAGCAGCGCTCATGGT
GGGGCAGCGCCTCACAACCTCCGTCATGTGCTGTGACTGCTTGTAGATGGCCATGGCGCG


In [ ]:
#figure out the best way to encode this information...

# PAM searching


In [82]:
def PAM_finder(seq, PAM):
    """ 
    Finds indeces of PAM sequences
    Formatting: e.g. "NGG"
    N = [A|T|C|G]
    R = [A|G]
    Y = [C|T]
    S = [G|C]
    W = [A|T]
    K = [G|T]
    M = [A|C]
    B = [C|G|T]
    D = [A|G|T]
    H = [A|C|T]
    V = [A|C|G]
    """
    PAM_dict = {"A":"A",
                "G":"G",
                "C":"C",
                "T":"T",
                "N":"[A|T|C|G]",
                "R":"[A|G]",
                "Y":"[C|T]",
                "S":"[G|C]",
                "W":"[A|T]",
                "K":"[G|T]",
                "M":"[A|C]",
                "B":"[C|G|T]",
                "D":"[A|G|T]",
                "H":"[A|C|T]",
                "V":"[A|C|G]"}

    PAM_new = ""
    for i in PAM:
        PAM_new += PAM_dict[i]

    p = re.compile(PAM_new)

    iterator = p.finditer(seq)
    PAM_holder = []
    for match in iterator:
        PAM_holder.append(match.span())

    return PAM_holder

In [86]:
seq = p53_formatted.iloc[0]['wt_w_context']
print(seq)
PAM = "NRCH"
PAM_holder = PAM_finder(seq, PAM)

for i in PAM_holder:
    print(seq[i[0]:i[1]])

CCTTTCTTGCGGAGATTCTCTTCCTCTGTGCGCCGGTCTCTCCCAGGACAGGCACAAACACGCACCTCAAAGCTGTTCCGTCCCAGTAGATTACCACTACTCAGGATAGGAAAAGAGAAGC
CGCC
GACA
GGCA
AACA
CGCA
AGCT
TACC
TACT


# Error checking


In [4]:
from Bio.Align import PairwiseAligner
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

In [5]:
def sensor_outcome_corrected(p53_lib, i):
    #p53_lib = df containing pegRNA + target sequence + other info
    #i = index

    sensor_true = p53_lib.iloc[i]['target sequence']

    #let's quantify the homology overhang as well...
    PAM_strand = p53_lib.iloc[i]['PAM strand']
    var_type = p53_lib.iloc[i]['variant_type']


    if PAM_strand=='+':
        sensor = Bio.Seq.Seq(p53_lib.iloc[i]['target sequence']).reverse_complement()

    elif PAM_strand=='-':
        sensor = p53_lib.iloc[i]['target sequence']

    pbs_length = p53_lib.iloc[i]['PBS length']
    pbs = p53_lib.iloc[i]['PBS_RTT_5to3'][-pbs_length:]
    rtt = p53_lib.iloc[i]['PBS_RTT_5to3'][:-(pbs_length-1)] #FIXING [:-0] indexing issue...

    #pbs_rtt = p53_lib.iloc[i]['PBS_RTT_5to3']

    pbs_find = sensor.find(pbs)

    #first compute the portion before the mutation for computing the right flank
    for kk, val in enumerate(reversed(rtt)):
        idx = pbs_find-kk
        if val!= sensor[idx]: #find first mismatch
            mut_start=kk
            break
        else:
            continue


    try: mut_start
    except: #case where's theres no identifiable change (because of homology overhang being too short/matching coincidentally...)
        return sensor_true, sensor_true, 0, "no homology overhang"

    #alignments = aligner.align(pbs, sensor)
    #print(alignments[0])

    #alignments = aligner.align(rtt, sensor)
    #print(alignments[0])

    #now we have the right flank...
    right_flank = rtt[-mut_start:-1] + sensor[pbs_find:]

    #alignments = aligner.align(right_flank, sensor)
    #print(alignments[0])
    
    if var_type in ['SNP', 'ONP']:

        matching_arr = np.zeros(len(rtt[:-mut_start]))

        for kk, val in enumerate(reversed(rtt[:-mut_start])):
            idx = pbs_find-mut_start-kk

            if val== sensor[idx]: #find regions of homology
                matching_arr[kk]+=1
                
            else:
                continue
        
        #based on values in match array, determine where mutation vs. homology overhang is...
        end_mut = np.where(matching_arr==0)[0][-1]

        mut_allele = rtt[-(mut_start+end_mut+1):-mut_start]
        homology_overhang = rtt[:-(mut_start+end_mut+1)]
        #this formulation will NOT catch situations where the homology overhang doesn't match???????
        #(though this shouldn't be an issue for SNPs/ONPs)

        left_flank = sensor[:(pbs_find-(len(rtt)-1))]

    elif var_type=='DEL':

        #find the size of the deletion by shifting and checking for matches...
        #will allow for detection of insufficient homology overhang...
        length_remaining = len(rtt[:-mut_start])
        homology_overhang = rtt[:-mut_start]
        mut_allele = ''

        start_idx = pbs_find-mut_start
        for i in range(len(sensor[:start_idx])): #check rest of the sensor...
        
            #sliding window to find size of deletion/perfect match...
            sensor_a = sensor[start_idx-i-length_remaining:start_idx-i]

            if sensor_a==homology_overhang: #once match is found, record and break...
                size_del = i
                break
        
            else:
                continue

        #error where the full homology overhang region not included in sensor...
        try: size_del
        except: 
            return sensor_true, sensor_true, len(homology_overhang), 'sensor too short' #no excepted editing, return WT sensor

        left_flank = sensor[:start_idx-size_del-length_remaining]

    elif var_type=='INS':

        length_remaining = len(rtt[:-mut_start])
        start_idx = pbs_find-mut_start+1

        for i in range(length_remaining):
            rtt_region = rtt[:-(mut_start+i)]
            sensor_a = sensor[start_idx-len(rtt_region):start_idx]

            #print('rt: ' + rtt_region)
            #print('s: ' + sensor_a)

            if sensor_a==rtt_region:
                mut_allele = rtt[-(mut_start+i):-mut_start]
                break
            else:
                continue

        #in case of no homology overhang...
        try: mut_allele
        except: 
            return sensor_true, sensor_true, 0, 'no homology overhang' #no excepted editing, return WT sensor

        homology_overhang = sensor_a
        left_flank = sensor[:start_idx-len(rtt_region)]

    edited_sensor = left_flank + homology_overhang + mut_allele + right_flank      

    if PAM_strand=='+':
        edited_sensor_true = str(Bio.Seq.Seq(edited_sensor).reverse_complement())

    elif PAM_strand=='-':
        edited_sensor_true=edited_sensor



    #alignments = aligner.align(rtt[-mut_start:], sensor)
    #print(alignments[0])

    #alignments = aligner.align(right_flank, sensor)
    #print(alignments[0])

    #alignments = aligner.align(homology_overhang, sensor)
    #print(alignments[0])

    #alignments = aligner.align(edited_sensor, sensor)
    #print(alignments[0])

    #alignments = aligner.align(edited_sensor_true, sensor_true)
    #print(alignments[0])


    return sensor_true, edited_sensor_true, len(homology_overhang), 'no error'

In [ ]:

p53_lib = ....

#running it for every mutation in the p53 library
edit_sens = []
homo = []
idx_holder = []
classif = []
#del_idx = p53_del.index

#ins_idx = p53_lib[p53_lib['variant_type']=='SNP'].index

error = []
for i in range(len(p53_lib)):
    try:
        s, s_e, homo2, m = sensor_outcome_corrected(p53_lib, i)
        idx_holder.append(i)
        edit_sens.append(s_e)
        homo.append(homo2)
        classif.append(m)
    
    except:
        error.append(i)